In [47]:
# %matplotlib widget

from __future__ import annotations


import matplotlib.colors as mcolors
from test_utilities import test, TestDict

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 11: Reactor ---</h2><p>You hear some loud beeping coming from a hatch in the floor of the factory, so you decide to check it out. Inside, you find several large electrical conduits and a ladder.</p>
<p>Climbing down the ladder, you discover the source of the <span title="The beeping is unrelated to the issue with the server rack. The reactor is just hungry.">beeping</span>: a large, toroidal reactor which powers the factory above. Some Elves here are hurriedly running between the reactor and a nearby server rack, apparently trying to fix something.</p>
<p>One of the Elves notices you and rushes over. "It's a good thing you're here! We just installed a new <em>server rack</em>, but we aren't having any luck getting the reactor to communicate with it!" You glance around the room and see a tangle of cables and devices running from the server rack to the reactor. She rushes off, returning a moment later with a list of the devices and their outputs (your puzzle input).</p>
<p>For example:</p>
<pre><code>aaa: you hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out
</code></pre>
<p>Each line gives the name of a device followed by a list of the devices to which its outputs are attached. So, <code>bbb: ddd eee</code> means that device <code>bbb</code> has two outputs, one leading to device <code>ddd</code> and the other leading to device <code>eee</code>.</p>
<p>The Elves are pretty sure that the issue isn't due to any specific device, but rather that the issue is triggered by data following some specific <em>path</em> through the devices. Data only ever flows from a device through its outputs; it can't flow backwards.</p>
<p>After dividing up the work, the Elves would like you to focus on the devices starting with the one next to you (an Elf hastily attaches a label which just says <em><code>you</code></em>) and ending with the main output to the reactor (which is the device with the label <em><code>out</code></em>).</p>
<p>To help the Elves figure out which path is causing the issue, they need you to find <em>every</em> path from <code>you</code> to <code>out</code>.</p>
<p>In this example, these are all of the paths from <code>you</code> to <code>out</code>:</p>
<ul>
<li>Data could take the connection from <code>you</code> to <code>bbb</code>, then from <code>bbb</code> to <code>ddd</code>, then from <code>ddd</code> to <code>ggg</code>, then from <code>ggg</code> to <code>out</code>.</li>
<li>Data could take the connection to <code>bbb</code>, then to <code>eee</code>, then to <code>out</code>.</li>
<li>Data could go to <code>ccc</code>, then <code>ddd</code>, then <code>ggg</code>, then <code>out</code>.</li>
<li>Data could go to <code>ccc</code>, then <code>eee</code>, then <code>out</code>.</li>
<li>Data could go to <code>ccc</code>, then <code>fff</code>, then <code>out</code>.</li>
</ul>
<p>In total, there are <code><em>5</em></code> different paths leading from <code>you</code> to <code>out</code>.</p>
<p><em>How many different paths lead from <code>you</code> to <code>out</code>?</em></p>
</article>


In [48]:
import re
from PrettyPrint import PrettyPrintTree

tests: list[TestDict] = [
    {
        "name": "Example",
        "s": """
            aaa: you hhh
            you: bbb ccc
            bbb: ddd eee
            ccc: ddd eee fff
            ddd: ggg
            eee: out
            fff: out
            ggg: out
            hhh: ccc fff iii
            iii: out
        """,
        "expected": 5,
    },
]


class Graph:
    def __init__(self, s: str) -> None:
        nodes = [re.findall(r"[a-z]+", l) for l in s.strip().splitlines()]
        self.graph = {}

        for node, *tos in nodes:
            self.graph[node] = tos

    def path_count_from_you_to_out(self) -> int:
        def dfs(node: str) -> int:
            if node == "out":
                return 1
            return sum(dfs(child) for child in self.graph[node])

        return dfs("you")

    def __str__(self, start_node: str = "you") -> str:
        pt = PrettyPrintTree(lambda x: self.graph.get(x, []), lambda x: str(x), return_instead_of_print=True)  # type: ignore
        return pt(start_node)  # type: ignore


@test(tests=tests)
def part_I(s: str) -> int:
    g = Graph(s)
    return g.path_count_from_you_to_out()


Test Example passed, for part_I.
Success


In [49]:
with open("../input/day11.txt") as f:
    puzzle = f.read()

print(f"Part I: {part_I(puzzle)}")

Part I: 708


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>708</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2></article>


In [50]:
from functools import cache


tests: list[TestDict] = [
    {
        "name": "Example",
        "s": """
            svr: aaa bbb
            aaa: fft
            fft: ccc
            bbb: tty
            tty: ccc
            ccc: ddd eee
            ddd: hub
            hub: fff
            eee: dac
            dac: fff
            fff: ggg hhh
            ggg: out
            hhh: out
        """,
        "expected": 2,
    },
]


class GraphII(Graph):
    def path_count_from_svr_to_out(self) -> int:
        @cache
        def dfs(node: str, dac, fft) -> int:
            if node == "out":
                return 1 if dac and fft else 0

            if node == "dac":
                dac = True
            elif node == "fft":
                fft = True

            return sum(dfs(child, dac, fft) for child in self.graph[node])

        return dfs("svr", False, False)

    def __str__(self, start_node: str = "svr") -> str:
        return super().__str__(start_node)


@test(tests=tests)
def part_II(s: str) -> int:
    g = GraphII(s)
    return g.path_count_from_svr_to_out()


Test Example passed, for part_II.
Success


<link href="style.css" rel="stylesheet"></link>


In [51]:
print(f"Part II: {part_II(puzzle)}")

Part II: 545394698933400


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>545394698933400</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
